<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/Conet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conet: Collaborative cross networks for cross-domain recommendation
# G Hu, Y Zhang, Q Yang
# Hu, G., Zhang, Y., & Yang, Q. (2018, October). Conet: Collaborative cross networks for cross-domain recommendation. In Proceedings of the 27th ACM international conference on information and knowledge management (pp. 667-676).

In [ ]:
import pandas as pd
import numpy as np
!pip install surprise

     |████████████████████████████████| 11.8 MB 5.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619424 sha256=14631aa508b4aff3532937e01d1e4c03dcd66e1dccdcb0c1fd0c8b2077ab8655
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


## Data Reading

In [ ]:
# Amazon books Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Books.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_books = pd.read_csv(infile)
  amazon_books = amazon_books[['user_id', 'item_id', 'rating']]
  amazon_books = amazon_books.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_books.columns
amazon_books.head()

,userId,itemId,rating
0,A1REUF3A1YCPHM,1713353,5
1,A1YRBRK2XM5D5,1713353,5
2,A1V8ZR5P78P4ZU,1713353,5
3,A2ZB06582NXCIV,1713353,5
4,ACPQVNRD3Z09X,1713353,5


In [ ]:
# Amazon Movies Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Movies.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_movies = pd.read_csv(infile)
  amazon_movies = amazon_movies[['user_id', 'item_id', 'rating']]
  amazon_movies = amazon_movies.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_movies.columns
amazon_movies.head()

,userId,itemId,rating
0,A2VHSG6TZHU1OB,1527665,5
1,A23EJWOW1TLENE,1527665,5
2,A1KM9FNEJ8Q171,1527665,5
3,A38LY2SSHVHRYB,1527665,4
4,AHTYUW2H1276L,1527665,5


In [ ]:
# Amazon Clothes Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Clothing.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_clothes = pd.read_csv(infile)
  amazon_clothes = amazon_clothes[['user_id', 'item_id', 'rating']]
  amazon_clothes = amazon_clothes.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_clothes.columns
amazon_clothes.head()

,userId,itemId,rating
0,A3OT9BYASFGU2X,871167042,4
1,A28GK1G2KDXHRP,871167042,5
2,A3NFXFEKW8OK0E,871167042,5
3,A3I6G5TKBVJEK9,871167042,5
4,A1A7Y1M8AJWNZ8,871167042,5


In [ ]:
# Amazon Video Games Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Video_Games.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_vg = pd.read_csv(infile)
  amazon_vg = amazon_vg[['user_id', 'item_id', 'rating']]
  amazon_vg = amazon_vg.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_vg.columns
amazon_vg.head()

In [ ]:
# MovieLens Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/movielens_ratings.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  movielens = pd.read_csv(infile)
movielens = movielens[['userId', 'movieId', 'rating']]
movielens = movielens.rename(columns={"movieId": "itemId"})
movielens.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
# BookCrossing Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/BX_Book_Ratings.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  bookcrossing = pd.read_csv(infile)
bookcrossing = bookcrossing[['User-ID', 'ISBN', 'Book-Rating']]
bookcrossing = bookcrossing.rename(columns={"User-ID":"userId", "ISBN":"itemId", "Book-Rating": "rating"})
bookcrossing.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
# Tripadvisor Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tripadvisor = pd.read_csv(infile)
tripadvisor = tripadvisor[['username', 'taObject', 'rating']]
tripadvisor = tripadvisor.rename(columns={"username":"userId", "taObject":"itemId"})
tripadvisor.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
source_user_ratings = amazon_books[0:10000]
target_user_ratings = amazon_movies[0:10000]

In [ ]:
print(source_user_ratings.shape)
print(target_user_ratings.shape)

(10000, 3)
(10000, 3)


In [ ]:
# creation of target rating matx for matx factorization(MF)
rs = source_user_ratings.pivot_table(index='userId',columns='itemId',values='rating')
print(rs.shape)
# creation of target rating matx for matx factorization(MF)
rt = target_user_ratings.pivot_table(index='userId',columns='itemId',values='rating')
print(rt.shape)

(9702, 67)
(9788, 70)


In [ ]:
rs

itemId,000047715X,000105001X,000171287X,000211495X,000216132X,1046519,1047566,1052292,1061240,13765,1381733,1384198,1472933,1473123,1477889,1711296,1712764,1712799,1712845,1713205,1713353,1714384,1714538,1714716,1716069,1720279,1720392,1839225,1839233,1840266,1840851,1841572,1844423,1857290,1921517,1932349,1937987,1945424,1950061,1954563,1983008,2000318,2005018,2005263,2005379,2005395,2006448,2006995,2007649,2007908,2007916,2008343,2008572,2008599,2051850,2103826,2111306,2112256,2117010,2117088,2152525,2153211,2153971,2160595,2163578,230022,913154
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A0334855HN6E38CXWXZR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0520969JGW2CZRMEZMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0790722OCX87RKL2J3T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100ATCZ8R81KG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A101M64MW6EH3O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZVK41090G5LI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZVVRNCE2F0ZY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZWV6VDHSTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating necessary matrices Is, It Rs Rt U

In [ ]:
# creting mask matrix Rs & Rt
rs_n = rs.replace(np.nan, 0)
rt_n = rt.replace(np.nan, 0)
rs_m = rs_n > 0
rt_m = rt_n > 0
rs_m = rs_m.replace(True, 1)
rs_m = rs_m.replace(False, 0)
rt_m = rt_m.replace(True, 1)
rt_m = rt_m.replace(False, 0)
Rs = rs_m
Rt = rt_m
print(Rs)
print(Rt)

itemId                000047715X  000105001X  ...  230022  913154
userId                                        ...                
A0334855HN6E38CXWXZR         0.0         0.0  ...     0.0     0.0
A0520969JGW2CZRMEZMM         0.0         0.0  ...     0.0     0.0
A0790722OCX87RKL2J3T         0.0         0.0  ...     0.0     0.0
A100ATCZ8R81KG               0.0         0.0  ...     0.0     0.0
A101M64MW6EH3O               0.0         0.0  ...     0.0     0.0
...                          ...         ...  ...     ...     ...
AZVK41090G5LI                0.0         0.0  ...     0.0     0.0
AZVVRNCE2F0ZY                0.0         0.0  ...     0.0     0.0
AZWV6VDHSTION                0.0         0.0  ...     0.0     0.0
AZY870M82TGEI                0.0         0.0  ...     0.0     0.0
AZYH2ZK8UE1YY                0.0         0.0  ...     0.0     0.0

[9702 rows x 67 columns]
itemId                000503860X  061524226X  ...  767809726  767813421
userId                                      

In [ ]:
# create Is and It items array and common users U
Is = np.array(source_user_ratings.itemId)
It = np.array(target_user_ratings.itemId)
U = np.intersect1d(np.array(source_user_ratings['userId']), np.array(target_user_ratings['userId']))
Is

array(['1713353', '1713353', '1713353', ..., '1844423', '1844423',
       '1844423'], dtype=object)

In [ ]:
U

## Matrix Factorization

In [ ]:
raw_t= Rt
raw_s= Rs

In [ ]:
import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [ ]:
Rs = numpy.array(Rs)
# N: num of User
Ns = len(Rs)
# M: num of Movie
Ms = len(Rs[0])
# Num of Features
K = 3

 
Ps = numpy.random.rand(N,K)
Qs = numpy.random.rand(M,K)

 

nP, nQ = matrix_factorization(Rs, Ps, Qs, K)

nR = numpy.dot(nP, nQ.T)